In [2]:
import numpy as np
import cv2
import os
import time
from matplotlib import pyplot as plt
from tqdm import tqdm
import glob
frames_dir = 'preprocessed-frames'
output_dir = 'bg-detection-frames'

In [12]:
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [ ]:
bg_subtractor = cv2.createBackgroundSubtractorKNN(history=300, detectShadows=False)

In [ ]:
for i in tqdm(range(250, 350)):
    frame = cv2.imread(f"{frames_dir}/frame{i}.jpg")
    img_sub = bg_subtractor.apply(frame)
    
    #edged = cv2.Canny(img_sub, 30, 200) 
    contours, _ = cv2.findContours(
        img_sub, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE
    )
    
    
    for c in contours:
        (x, y, w, h) = cv2.boundingRect(c)
        area = cv2.contourArea(c)
        aspect_ratio = float(h) / w
        # if area > 10:
        #     print(area)

        # if 10000 <= area <= 50000 and 0.2 <= aspect_ratio <= 0.5:
        #     print("person detected in frame: ", i)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
    cv2.imwrite(f"{output_dir}/frame{i}.jpg", frame)    


In [ ]:
#print(areas)

In [13]:
CONTOUR_WIDTH = (70, 300)
CONTOUR_HEIGHT = (70, 300)

In [ ]:
def drawBoundingBoxes(contours, frame) -> tuple[list, list]:
    """
    Draw bounding boxes around detected vehicles and return their center c
    return frameoordinates

    Args:
        contours (list): a list of contours
        frame (np.ndarray): the frame

    Returns:
        detectedVehicles (list): a list of detected vehicles
        boundingBoxes (list): a list of bounding boxes
    """

    boundingBoxes = []

    for c in contours:
        (x, y, w, h) = cv2.boundingRect(c)
        contour_valid = (
            (w >= CONTOUR_WIDTH[0])
            and (h >= CONTOUR_HEIGHT[0])
            and (w <= CONTOUR_WIDTH[1])
            and (h <= CONTOUR_HEIGHT[1])
        )

        if not contour_valid:
            continue

        # draw bounding boxes
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        boundingBoxes.append((x, y, w, h))

    return boundingBoxes

In [ ]:
#contours, _ = cv2.findContours(
#            filteredImage, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE
#        )

#boundingBoxes = drawBoundingBoxes(contours, frame)


In [ ]:
plt.imshow(frame)
plt.show()

In [3]:
bg_subtractor = cv2.createBackgroundSubtractorKNN(history=300, detectShadows=False)
output_dir="bg-detection-frames-detect-whole-object"

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for i in tqdm(range(250, 350)):
    frame = cv2.imread(f"{frames_dir}/frame{i}.jpg")
    img_sub = bg_subtractor.apply(frame)
    
    #edged = cv2.Canny(img_sub, 30, 200) 

    # threshold if it is bigger than 240 pixel is equal to 255 if smaller pixel is equal to 0
    # create binary image , it contains only white and black pixels (since shadow are grey)
    #ret , treshold = cv2.threshold(img_sub.copy(), 120, 255,cv2.THRESH_BINARY)
    
    #  dilation expands or thickens regions of interest in an image.
    dilated = cv2.dilate(img_sub,cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3)),iterations = 2)

    contours, _ = cv2.findContours(
        #RETR_EXTERNAL, prende solo i contorni esternie non quelli interni
        #RETR_TREE: Rileva tutti i contorni e costruisce una gerarchia (inclusi i contorni interni).
        dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
    )
    
    
    for c in contours:
         # if area exceed certain value then draw bounding boxes
        if cv2.contourArea(c) > 50:
            (x, y, w, h) = cv2.boundingRect(c)
            area = cv2.contourArea(c)
            cv2.rectangle(frame, (x,y), (x+w, y+h), (255, 255, 0), 2)
            aspect_ratio = float(h) / w
            
    
    cv2.imwrite(f"{output_dir}/frame{i}.jpg", frame)    


  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:21<00:00,  4.61it/s]


In [33]:
bg_subtractor = cv2.createBackgroundSubtractorKNN(history=300, detectShadows=False)

In [34]:
output_dir="bg-detection-frames-detect-whole-object-freeman"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [35]:
for i in tqdm(range(250, 350)):
    frame = cv2.imread(f"{frames_dir}/frame{i}.jpg")
    img_sub = bg_subtractor.apply(frame)
    
    #edged = cv2.Canny(img_sub, 30, 200) 

    # threshold if it is bigger than 240 pixel is equal to 255 if smaller pixel is equal to 0
    # create binary image , it contains only white and black pixels (since shadow are grey)
    #ret , treshold = cv2.threshold(img_sub.copy(), 120, 255,cv2.THRESH_BINARY)
    
    #  dilation expands or thickens regions of interest in an image.
    dilated = cv2.dilate(img_sub,cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3)),iterations = 2)

    contours, _ = cv2.findContours(
        #RETR_EXTERNAL, prende solo i contorni esternie non quelli interni
        #RETR_TREE: Rileva tutti i contorni e costruisce una gerarchia (inclusi i contorni interni).
        dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
    )
    
    
    for c in contours:
         # if area exceed certain value then draw bounding boxes
        if cv2.contourArea(c) > 50:
            
            (x,y,w,h) = cv2.boundingRect(c)
            cv2.rectangle(frame, (x,y), (x+w, y+h), (255, 255, 0), 2)
            aspect_ratio = float(h) / w
            '''
            '''
            # Applica il Freeman Chain Code
            chain_code = []
            contour_points = c.squeeze(axis=1)  # Ottieni i punti del contorno
            
            for j in range(len(contour_points) - 1):
                dx = contour_points[j + 1][0] - contour_points[j][0]
                dy = contour_points[j + 1][1] - contour_points[j][1]

                # Direzione secondo Freeman (8 direzioni)
                if dx == 1 and dy == 0:
                    chain_code.append(0)
                elif dx == 1 and dy == -1:
                    chain_code.append(1)
                elif dx == 0 and dy == -1:
                    chain_code.append(2)
                elif dx == -1 and dy == -1:
                    chain_code.append(3)
                elif dx == -1 and dy == 0:
                    chain_code.append(4)
                elif dx == -1 and dy == 1:
                    chain_code.append(5)
                elif dx == 0 and dy == 1:
                    chain_code.append(6)
                elif dx == 1 and dy == 1:
                    chain_code.append(7)
            
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 255, 0), 2)
            
    cv2.imwrite(f"{output_dir}/frame{i}.jpg", frame)    


100%|██████████| 100/100 [00:20<00:00,  4.86it/s]


TODO: riguardare il freeman e detectare anche quando è fermo